In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

#from fl_mnist_implementation_tutorial_utils import *

Mounted at /content/drive


In [ ]:
###    For the given path, get the List of all files in the directory tree
def getListOfFiles(dirName):
    #RESIZE=224
    # create a list of file and sub directories
    # names in the given directory
    listOfFile = os.listdir(dirName)
    allFiles = list()

    #read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    # Iterate over all the entries
    #print(entry)
    for entry in listOfFile:
        print(entry)
        # Create full path
        fullPath = os.path.join(dirName, entry)

        if (entry == 'TCGA.94.A5I4'):
          continue

        if (entry == 'TCGA.43.6770'):
          continue
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):

            allFiles = allFiles + getListOfFiles(fullPath)

        else:
            #img = read(fullPath)

            #img = cv2.resize(img, (RESIZE,RESIZE))
            im_gray = cv2.imread(fullPath, cv2.IMREAD_GRAYSCALE)
            image = np.array(im_gray).flatten()
            # scale the image to [0, 1] and add to list

            allFiles.append(image/255)
        #print(entry)

    return allFiles


benign_train = np.array(getListOfFiles('/content/drive/My Drive/Datasets/Tiles/LUSC1'))
malign_train = np.array(getListOfFiles('/content/drive/My Drive/Datasets/Tiles/LUAD1'))
#benign_valid = np.array(getListOfFiles('C:/DataSets/breast/validation/benign'))
#malign_valid = np.array(getListOfFiles('C:/DataSets/breast/validation/Malignant_validv2'))


# Print number of files for train/test datasets
print('There are %d total Benign Train files.' % len(benign_train))
print('There are %d total Malignant Train files.' % len(malign_train))



Streaming output truncated to the last 5000 lines.
2508.png
2509.png
2510.png
2511.png
2512.png
2529.png
2530.png
2531.png
2532.png
2533.png
2534.png
2535.png
2536.png
2537.png
2538.png
2539.png
2540.png
2541.png
2542.png
2543.png
2544.png
2545.png
2546.png
2547.png
2548.png
2549.png
2550.png
2551.png
2552.png
2553.png
2554.png
2555.png
2556.png
2557.png
2558.png
2559.png
2560.png
2561.png
2570.png
2571.png
2572.png
2573.png
2574.png
2575.png
2576.png
2577.png
2578.png
2579.png
2580.png
2581.png
2582.png
2583.png
2584.png
2585.png
2586.png
2587.png
2588.png
2589.png
2590.png
2591.png
2592.png
2593.png
2594.png
2595.png
2596.png
2597.png
2598.png
2599.png
2600.png
2601.png
2602.png
2603.png
2618.png
2619.png
2620.png
2621.png
2622.png
2623.png
2624.png
2625.png
2626.png
2627.png
2628.png
2629.png
2630.png
2631.png
2632.png
2633.png
2634.png
2635.png
2636.png
2637.png
2638.png
2639.png
2640.png
2641.png
2642.png
2643.png
2644.png
2645.png
2646.png
2647.png
2648.png
2649.png
2650.png
2651

In [ ]:
#----Create numpy array of 'zeros' and 'ones' for labelling benign/malignant images resp.
benign_train_label = np.zeros(len(benign_train))
malign_train_label = np.ones(len(malign_train))
#benign_train_label = (-1)*benign_train_label2

X_data = np.concatenate((benign_train, malign_train), axis = 0)
Y_data = np.concatenate((benign_train_label, malign_train_label), axis = 0)
#X_test = np.concatenate((benign_valid, malign_valid), axis = 0)
#Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

s = np.arange(X_data.shape[0])
np.random.shuffle(s)
X_data = X_data[s] #Re-index
Y_data = Y_data[s]

#binarize the labels
#lb = LabelBinarizer()
#label_list = lb.fit_transform(Y_data)

#split data into training and test set


x_train, x_test1, y_train, y_test1 = train_test_split(X_data,
                                                    Y_data,
                                                    test_size=0.3,
                                                    random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test1,
                                                    y_test1,
                                                    test_size=0.5,
                                                    random_state=42)


In [ ]:
from keras.optimizers import SGD
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("sigmoid"))
        return model


lr = 0.01
comms_round = 100
loss='sparse_categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=lr)



"\n\n# Instantiate the Pre_trained Convnet\nconv_base = ResNet50(weights='imagenet',\n                  include_top=False,\n                  input_shape=(224, 224, 3))\n\nconv_base.summary()\n\ndef build_model(backbone, lr):\n    model = Sequential()\n    model.add(backbone)\n    model.add(layers.GlobalAveragePooling2D())\n    model.add(layers.Dropout(0.5))\n    model.add(layers.BatchNormalization())\n    model.add(layers.Dense(2, activation='sigmoid'))\n\n    model.compile(\n        loss='binary_crossentropy',\n        optimizer=Adam(lr=lr),\n        metrics=['acc']\n        #metrics=['acc','val_acc','loss','val_loss']\n    )\n    return model\n\n\n\nmodel = build_model(conv_base ,lr = 1e-4)\nmodel.summary()\n"

In [ ]:
smlp_local = SimpleMLP()
local_model = smlp_local.build(961, 2)
local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)

#model = build
BATCH_SIZE = 16

history = local_model.fit(
    x_train, y_train, batch_size=BATCH_SIZE,
    #datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=50,
    validation_data=(x_val, y_val),
    #callbacks=plot_loss_1
    #callbacks=[learn_control, checkpoint]
    #callbacks=[checkpoint]
)

Epoch 1/50
75/75 [==============================] - 1s 7ms/step - loss: 0.5509 - accuracy: 0.8002 - val_loss: 0.5520 - val_accuracy: 0.8031
Epoch 2/50
75/75 [==============================] - 0s 5ms/step - loss: 0.5206 - accuracy: 0.8068 - val_loss: 0.5522 - val_accuracy: 0.8108
Epoch 3/50
75/75 [==============================] - 0s 5ms/step - loss: 0.4877 - accuracy: 0.8167 - val_loss: 0.5140 - val_accuracy: 0.7992
Epoch 4/50
75/75 [==============================] - 0s 5ms/step - loss: 0.4723 - accuracy: 0.8183 - val_loss: 0.4992 - val_accuracy: 0.8224
Epoch 5/50
75/75 [==============================] - 0s 4ms/step - loss: 0.4437 - accuracy: 0.8431 - val_loss: 0.5915 - val_accuracy: 0.7761
Epoch 6/50
75/75 [==============================] - 0s 4ms/step - loss: 0.4272 - accuracy: 0.8439 - val_loss: 0.4698 - val_accuracy: 0.8301
Epoch 7/50
75/75 [==============================] - 0s 7ms/step - loss: 0.4026 - accuracy: 0.8621 - val_loss: 0.4710 - val_accuracy: 0.8108
Epoch 8/50
75/75 [==

Federated Members (clients) as Data Shards

In [ ]:
def create_clients(image_list, label_list, num_clients=2, initial='wwwwwwwwwwwwwwwwwwwwwwwwww'):
    ''' return: a dictionary with keys clients' names and value as
                data shards - tuple of images and label lists.
        args:
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1

    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    print(client_names)
    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

#create clients
clients = create_clients(X_train, y_train, num_clients=2, initial='client')

['client_1', 'client_2']


Processing and batching clients’ and test data

In [ ]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)


#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

Creating the Multi Layer Perceptron (MLP) model

In [ ]:
from keras.optimizers import SGD
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("sigmoid"))
        return model


lr = 0.01
comms_round = 100
loss='sparse_categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=lr)

Model Aggregation (Federated Averaging)

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad




In [ ]:
def test_model(X_test, y_test,  model, comm_round):
    #cce = tf.keras.losses.SparseCategoricalCrossentropy()
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test, batch_size=100)
    #logits = (model.predict(X_test) > 0.5).astype("float")
    #logits = model.predict(X_test)
    loss = cce(y_test, logits)
    #print(logits)
    #loss = cce(y_test)
    #acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(y_test, axis = 1))
    acc = accuracy_score(tf.argmax(logits, axis=1), y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

Federated Model Training


In [ ]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(961, 2)

global_acc_list = []
global_loss_list = []

#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(961, 2)
        local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, y_test, global_model, comm_round)




2/2 [==============================] - 0s 9ms/step
comm_round: 0 | global_acc: 82.081% | global_loss: 0.6163625121116638
2/2 [==============================] - 0s 8ms/step
comm_round: 1 | global_acc: 80.925% | global_loss: 0.6353694200515747
2/2 [==============================] - 0s 7ms/step
comm_round: 2 | global_acc: 80.925% | global_loss: 0.6263630390167236
2/2 [==============================] - 0s 7ms/step
comm_round: 3 | global_acc: 80.925% | global_loss: 0.6348424553871155
2/2 [==============================] - 0s 7ms/step
comm_round: 4 | global_acc: 81.503% | global_loss: 0.6174077987670898
2/2 [==============================] - 0s 8ms/step
comm_round: 5 | global_acc: 81.503% | global_loss: 0.6229887008666992
2/2 [==============================] - 0s 13ms/step
comm_round: 6 | global_acc: 81.503% | global_loss: 0.5995201468467712
2/2 [==============================] - 0s 7ms/step
comm_round: 7 | global_acc: 81.503% | global_loss: 0.6150895357131958
2/2 [==========================

SGD Vs Federated Averaging

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10)

SGD_model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)


In [ ]:
def non_iid_x(image_list, label_list, x=1, num_intraclass_clients=10):
        ''' creates x non_IID clients
        args:
            image_list: python list of images or data points
            label_list: python list of labels
            x: none IID severity, 1 means each client will only have one class of data
            num_intraclass_client: number of sub-client to be created from each none IID class,
            e.g for x=1, we could create 10 further clients by splitting each class into 10

        return - dictionary
            keys - clients's name,
            value - client's non iid 1 data shard (as tuple list of images and labels) '''

        non_iid_x_clients = dict()

        #create unique label list and shuffle
        unique_labels = np.unique(np.array(label_list))
        random.shuffle(unique_labels)

        #create sub label lists based on x
        sub_lab_list = [unique_labels[i:i + x] for i in range(0, len(unique_labels), x)]

        for item in sub_lab_list:
            class_data = [(image, label) for (image, label) in zip(image_list, label_list) if label in item]

            #decouple tuple list into seperate image and label lists
            images, labels = zip(*class_data)

            # create formated client initials
            initial = ''
            for lab in item:
                initial = initial + lab + '_'

            #create num_intraclass_clients clients from the class
            intraclass_clients = create_clients(list(images), list(labels), num_intraclass_clients, initial)

            #append intraclass clients to main clients'dict
            non_iid_x_clients.update(intraclass_clients)

        return non_iid_x_clients